In [2]:
from typing import Tuple, Union, List
import numpy as np
from sklearn.linear_model import LogisticRegression
import openml
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score

In [3]:
import pickle
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("data/test.csv")
df

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App,loan_status
0,8400,0.1797,303.56,81600,11.78,0,0,10,0,5126,...,0,1,0,1,0,0,1,1,0,0
1,35000,0.1298,1178.96,140000,27.93,5,0,29,0,51257,...,0,0,1,0,0,0,1,1,0,0
2,30000,0.0646,918.93,283000,5.22,0,1,25,0,43694,...,1,0,0,0,0,0,1,1,0,0
3,15000,0.2727,460.46,77000,10.38,1,1,14,1,7837,...,0,0,0,0,0,0,1,1,0,0
4,25000,0.2437,985.69,85000,12.33,0,0,10,1,64320,...,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9600,0.0819,301.68,70000,22.15,1,0,5,0,3327,...,0,1,0,0,0,0,1,1,0,1
96,40000,0.0819,814.70,161000,18.11,3,0,11,0,71158,...,0,0,1,0,0,0,1,1,0,1
97,21000,0.2534,620.58,77000,26.14,0,2,10,0,6265,...,0,0,1,0,0,0,0,1,1,1
98,10000,0.1614,352.27,65000,23.76,0,0,5,0,651,...,0,1,1,0,0,0,1,1,0,1


In [5]:
X_test = df.iloc[::,0:-1]
Y_test = df.iloc[::,-1]

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaler = scaler.fit(X_test)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
from os import listdir
from os.path import isfile, join
client1 = [f for f in listdir('model/client1') if isfile(join('model/client1', f))]
client2 = [f for f in listdir('model/client2') if isfile(join('model/client2', f))]
client3 = [f for f in listdir('model/client3') if isfile(join('model/client3', f))]
client4 = [f for f in listdir('model/client4') if isfile(join('model/client4', f))]
agg_models = [f for f in listdir('model/agg_models') if isfile(join('model/agg_models', f))]

In [25]:
client1

['client_1_model_0.sav',
 'client_1_model_1.sav',
 'client_1_model_2.sav',
 'client_1_model_3.sav']

In [26]:
client2

['client_2_model_0.sav',
 'client_2_model_1.sav',
 'client_2_model_2.sav',
 'client_2_model_3.sav']

In [27]:
client3

['client_3_model_0.sav',
 'client_3_model_1.sav',
 'client_3_model_2.sav',
 'client_3_model_3.sav']

In [28]:
client4

['client_4_model_0.sav',
 'client_4_model_1.sav',
 'client_4_model_2.sav',
 'client_4_model_3.sav']

In [29]:
agg_models

['agg_model_0.sav', 'agg_model_1.sav', 'agg_model_2.sav', 'agg_model_3.sav']

In [35]:
from os import listdir
from os.path import isfile, join
client1 = [f for f in listdir('model/client1') if isfile(join('model/client1', f))]
client2 = [f for f in listdir('model/client2') if isfile(join('model/client2', f))]
client3 = [f for f in listdir('model/client3') if isfile(join('model/client3', f))]
client4 = [f for f in listdir('model/client4') if isfile(join('model/client4', f))]
agg_models = [f for f in listdir('model/agg_models') if isfile(join('model/agg_models', f))]
import pickle
count = 0
for client1m in client1:
    c1_model = pickle.load(open(f'model/client1/{client1m}', 'rb'))
    c1_preds = c1_model.predict(X_test_scaled)
    print(f"-----------\033[1m CLIENT-1 \033[0m ---------")
    print(classification_report(Y_test, c1_preds))
    print("=================================================================")
    count +=1
for client1m in client2:
    c1_model = pickle.load(open(f'model/client2/{client1m}', 'rb'))
    c1_preds = c1_model.predict(X_test_scaled)
    print(f"-----------\033[1m CLIENT-2 \033[0m ---------")
    print(classification_report(Y_test, c1_preds))
    print("=================================================================")
    count +=1
for client1m in client3:
    c1_model = pickle.load(open(f'model/client3/{client1m}', 'rb'))
    c1_preds = c1_model.predict(X_test_scaled)
    print(f"-----------\033[1m CLIENT-3 \033[0m ---------")
    print(classification_report(Y_test, c1_preds))
    print("=================================================================")
    count +=1
for client1m in client4:
    c1_model = pickle.load(open(f'model/client4/{client1m}', 'rb'))
    c1_preds = c1_model.predict(X_test_scaled)
    print(f"-----------\033[1m CLIENT-4 \033[0m ---------")
    print(classification_report(Y_test, c1_preds))
    print("=================================================================")
    count +=1

----------- CLIENT-1  ---------
              precision    recall  f1-score   support

           0       0.80      0.70      0.74        50
           1       0.73      0.82      0.77        50

    accuracy                           0.76       100
   macro avg       0.76      0.76      0.76       100
weighted avg       0.76      0.76      0.76       100

----------- CLIENT-2  ---------
              precision    recall  f1-score   support

           0       0.86      0.64      0.74        50
           1       0.71      0.90      0.80        50

    accuracy                           0.77       100
   macro avg       0.79      0.77      0.77       100
weighted avg       0.79      0.77      0.77       100

